# Five Maternal Characteristics Determine the 12-month Course of Postpartum Depression Symptoms

### Objective: 
Postpartum depression is a heterogeneous disorder in phenotype and etiology. Characterizing the longitudinal course of depressive symptoms over the first year after birth and identifying variables that predict distinct symptom trajectories will expedite efficient mental health treatment planning.  

The purpose was to 
1. Determine 12-month trajectories of postpartum depressive symptoms
2. Identify characteristics that predict the trajectories
3. Provide a computational algorithm that predicts specific trajectory membership. 

### Analysis Methods:

The STATA/SAS plugin traj was used for estimating group-based trajectory models of SIGH-ADS29 total score during the first year postpartum. The number of trajectories and the shape of the pattern of change for each over time were determined from the comparison of Bayesian Information Criterion (BIC) and visual fit across all possible models.

After selecting a final model, we used the newly-created trajectory group membership variable to examine differences for demographic and other potential predictors across these categories.  We employed Pearson’s Chi-square test or Fisher’s exact test for categorical variables and one-way ANOVA for continuous variables. 

We created a multinomial logistic regression model with the trajectory group membership as the outcome to assess the capacity of baseline variables to predict one-year postpartum depression trajectories. Each demographic predictor was tested in an unadjusted multinomial logistic regression model. Predictors that were individually statistically significant at the 10% level were assessed in the multivariable model. Development of a final predictive model for trajectory membership employed 10-fold cross validation techniques using a penalized multinomial regression model through the R package CARET (Classification and Regression Training). Variables that remained statistical significant at the 10% level were kept in the reduced model. 
    
Missing covariates were handled by multiple imputation using the R package MICE (Multiple Imputation by Chained Equations). The model results were compared with results from complete case analysis. 
    
Post hoc analyses compared the two most severe trajectories with respect to the use of health services at baseline, month 6 and month 12 via a series of generalized linear mixed effect models.  In each model the use of health services was regressed on fixed trajectory groups, time, and the trajectory-by-time interaction terms (assuming a 10% level of significance for interaction terms) with an additional random intercept term. Analyses were performed using STATA 27 and R 3.2.0.

#### Step 1: Determine the trajectory patterns -STATA traj 

Tried 2,3 and 4 groups --> 3 groups gave the best BIC

At least one group has <5% of the sample if group number >4-->stopped testing at 4 groups

Then, determine the appropriate form of the groups
intercept only, linear, quadratic and cubic

<img src="https://raw.githubusercontent.com/amysheep/NUTrajectoryPredictionAnalysis/master/Graph-213_06142017.png">

#### Step 1 results: Three distinct trajectories of depressive symptoms were identified: 1) gradual remission (50.4%), 2) partial improvement (41.8%), and 3) chronic, non-remission (7.8%). 

PS: this is an unsupervised learning/clustering problem.

#### Step 2: Which factors are significantly associated with the trajactory patterns?

In [ ]:
### some data cleaning using tidyverse

library(caret)
library(tidyverse)
library(nnet)
library(broom)

data=read.csv("../Data/dataforsastraj06142017_out.csv",h=T)
diag=read.csv("../Data/diag.csv")


# merge comorbidity data from diag to create Anxiety Yes No variable
colnames(diag)[1]="id"
diag1=filter(diag,id%in%data$id)%>%select(id,preg,category,short_di)
anxid=filter(diag1,short_di%in%c("Social Phobia","Posttraumatic Stress Disorder","Panic Disorder With Agoraphobia","Panic Disorder Without Agoraphobia","Obsessive-Compulsive Disorder","Generalized Anxiety Disorder"),category=="Anxiety Disorders")%>%select(id,preg)

anxid=unique(anxid)
anxid$anx=1
dataall=left_join(data,anxid,by=c("id","preg"))%>%mutate(anx=ifelse(is.na(anx),0,anx))

# also recoded some of the rare categories in categorical variables to eliminate near-zero-variance issue
# code not show here

### Create Table 1

library(compareGroups)

t=compareGroups(X_traj_Group~.,data=t1data,include.miss=F)
t1=createTable(t,show.all = T,hide=c(hispanic="0",harm01="Never",marital01="Single or divorced",edu12="Some college or less"),hide.no = "no",digits=2)
t1 

#### Step 2 results:

<img src="https://raw.githubusercontent.com/amysheep/NUTrajectoryPredictionAnalysis/master/Tab1.png">

#### Step 3: Complete case multinomial logistic regression (n=417)

--Full model with p<0.1 factors from table 1 (10-fold cross validation R package `caret`-classification and regression training) 
    Note: Variables are tested for collinearity, near-zero-variance, outliers etc. before modelling.

--Penalized multinomial regression model (tuning parameter-weight decay:decay is the regularization parameter to avoid over-fitting)

--Use Cross validation to find the optimal tuning parameter that gives the best accuracy 

--Use upsampling within resampling for class imbalance (group 3 is way too low compared to 1 and 2)

In [ ]:
# specify type of resampling
train_control<- trainControl(method="cv", number=10,savePredictions = TRUE,sampling="up")

# fitting multinomial logistic regression -multinom function from nnet package
set.seed(06132017)
model<- train(X_traj_Group~edu12+anx+physab+adltph+parity+n_cmdx+epdstota+gasscore00, 
              data=t1data, trControl=train_control, method="multinom",
              na.action=na.omit,savePredictions = TRUE,
              tuneGrid=expand.grid(.decay=c(0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9)),trace=F)
# optimized model
finalmodel1 <- summary(model$finalModel)

# there are a few variables had p>0.1 in the above full model
# remove them and create a reduced model 

set.seed(061320171)
model2<- train(X_traj_Group~adltph+anx+parity+n_cmdx+gasscore00,
               data=t1data, trControl=train_control, method="multinom",
               na.action=na.omit,savePredictions = TRUE,
               tuneGrid=expand.grid(.decay=c(0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9)),trace=F)


# use relevel to get 2vs3 comparisons
t1data$re_group=relevel(t1data$X_traj_Group,ref="2")
model2a<- train(re_group~adltph+anx+parity+n_cmdx+gasscore00, 
                data=t1data, trControl=train_control, 
                method="multinom",na.action=na.omit,savePredictions = TRUE,
                tuneGrid=expand.grid(.decay=c(0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9)),trace=F)

# use Broom package to clean up output table and calculate ORs and 95%CIs 

library(broom)

tidy(model2$finalModel)->m2
tidy(model2a$finalModel)->m2a

table_func <- function(m){
  or <- round(exp(m$estimate),3)
  or_low <- round(exp(m$estimate-1.96*m$std.error),3)
  or_high <- round(exp(m$estimate+1.96*m$std.error),3)
  group <- m$y.level
  var <- m$term
  p <- round(m$p.value,3)
  out <- data.frame(cbind(group,var,or,or_low,or_high,p))
  return(out)
}

table_func(m2)[c(2:6,8:12),]
table_func(m2a)[c(2:6,8:12),]

# confusion matrix

filter(model2$pred,decay==0.7)%>%select(obs,pred)->cm
confusionMatrix(data=cm$pred,reference=cm$obs)


#### Step 3 results :
Accuracy was used to select the optimal model using the largest value.
The final value used for the model was decay = 0.7.

<img src="https://raw.githubusercontent.com/amysheep/NUTrajectoryPredictionAnalysis/master/fig2.png">

Note: Could also split the data into training/testing sets before model fitting. Did not show code here to save space.

The accuracy isn't that high.

#### Step 4: Model with multiple imputation (5 different sets of imputations) for missing covariates (R package `MICE`-multiple imputation by chained equations)

A. Inspecting the missing data

In [ ]:
library(mice)
mdpt <- md.pattern(select(t1data,edu12,anx,physab,adltph,parity,n_cmdx,epdstota,gasscore00))
colnames(mdpt)[9] <- "Number of missing Vars"
rownames(mdpt)[7] <- "Total Missing"
rownames(mdpt) <- paste0("N=",rownames(mdpt))
mdpt

<img src="https://raw.githubusercontent.com/amysheep/NUTrajectoryPredictionAnalysis/master/fig3.png">

-- There are six different patterns (n=417 complete data; n=57 missing number of chronic medical conditions; n=14 missing gas;
n=9 missing child and adult physical abuse; n=8 missing gas and chronic med conditions; n=2 missing child and adult physical abuse and chronic med conditions )

-- N miss=11 for child and adult physical abuse; N miss=22 for GAS; N=67 for number of chronic medical conditions

B. Create 5 imputated datasets 

In [ ]:
imp <- mice(select(t1data,X_traj_Group,edu12,anx,physab,adltph,parity,n_cmdx,epdstota,gasscore00),
            me=c("","","","logreg","logreg","","pmm","","pmm"))

C. Diagnostic checking 
(whether imputated values are plausible e.g.scores and counts are positive numbers)

In [ ]:
stripplot(imp,n_cmdx+adltph~.imp,pch=20,cex=1.2,layout=c(1,2))
stripplot(imp,physab+gasscore00~.imp,pch=20,cex=1.2,layout=c(1,2))

<img src="https://raw.githubusercontent.com/amysheep/NUTrajectoryPredictionAnalysis/master/fig4.png">

D. Analysis of imputed data (Full model with 8 predictors)

-- Within each imputed dataset we repeat fitting the prediction model with cross validation 

-- The final estimates are pooled from the 5 sets of estimates

-- EPDS is also a significant factor in addition to the 5 found in the data without imputation

-- Final reduced model

In [ ]:
fit1=train(X_traj_Group~edu12+anx+physab+adltph+parity+n_cmdx+epdstota+gasscore00, trControl=train_control, method="multinom",savePredictions = TRUE,tuneGrid=expand.grid(.decay=c(0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9)),trace=F,data=mice::complete(imp))
fit2=train(X_traj_Group~edu12+anx+physab+adltph+parity+n_cmdx+epdstota+gasscore00, trControl=train_control, method="multinom",savePredictions = TRUE,tuneGrid=expand.grid(.decay=c(0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9)),trace=F,data=mice::complete(imp,2))
fit3=train(X_traj_Group~edu12+anx+physab+adltph+parity+n_cmdx+epdstota+gasscore00, trControl=train_control, method="multinom",savePredictions = TRUE,tuneGrid=expand.grid(.decay=c(0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9)),trace=F,data=mice::complete(imp,3))
fit4=train(X_traj_Group~edu12+anx+physab+adltph+parity+n_cmdx+epdstota+gasscore00, trControl=train_control, method="multinom",savePredictions = TRUE,tuneGrid=expand.grid(.decay=c(0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9)),trace=F,data=mice::complete(imp,4))
fit5=train(X_traj_Group~edu12+anx+physab+adltph+parity+n_cmdx+epdstota+gasscore00, trControl=train_control, method="multinom",savePredictions = TRUE,tuneGrid=expand.grid(.decay=c(0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9)),trace=F,data=mice::complete(imp,5))
#The as.mira() function takes the results of repeated complete-data analysis stored as a list, and turns it into a mira object that can be pooled.
fit=as.mira(list((fit1$finalModel),(fit2$finalModel),(fit3$finalModel),(fit4$finalModel),(fit5$finalModel)))

# calculate pooled estimates
beta1=summary(fit1$finalModel)$coefficients
beta2=summary(fit2$finalModel)$coefficients
beta3=summary(fit3$finalModel)$coefficients
beta4=summary(fit4$finalModel)$coefficients
beta5=summary(fit5$finalModel)$coefficients

betaall=rbind(beta1,beta2,beta3,beta4,beta5)
beta2vs1=betaall[rownames(betaall)==2,]
beta3vs1=betaall[rownames(betaall)==3,]

# pooled beta estimate
grandm2vs1=apply(beta2vs1,2,mean)
grandm3vs1=apply(beta3vs1,2,mean)

# pooled se of beta 
se1=summary(fit1$finalModel)$standard.errors
se2=summary(fit2$finalModel)$standard.errors
se3=summary(fit3$finalModel)$standard.errors
se4=summary(fit4$finalModel)$standard.errors
se5=summary(fit5$finalModel)$standard.errors

seall=rbind(se1,se2,se3,se4,se5)
se2vs1=seall[rownames(seall)==2,]
se3vs1=seall[rownames(seall)==3,]

withinse2vs1=apply(se2vs1,2,mean)
withinse3vs1=apply(se3vs1,2,mean)

# calculate p-value
between2vs1=NULL
for (i in 1:9){
  t=sum((beta2vs1[,i]-grandm2vs1[i])^2)/(5-1)
  between2vs1=rbind(between2vs1,t)
}

between3vs1=NULL
for (i in 1:9){
  t=sum((beta3vs1[,i]-grandm3vs1[i])^2)/(5-1)
  between3vs1=rbind(between3vs1,t)
}

grandvar2vs1 <- withinse2vs1^2 + ((1 + (1/5)) * between2vs1)
grandse2vs1 <- sqrt(grandvar2vs1)
#grandse2vs1

grandvar3vs1 <- withinse3vs1^2 + ((1 + (1/5)) * between3vs1)
grandse3vs1 <- sqrt(grandvar3vs1)
#grandse3vs1

z2vs1=grandm2vs1/grandse2vs1
p2vs1=(1-pnorm(abs(z2vs1),0,1))*2
p2vs1=round(p2vs1,3)

z3vs1=grandm3vs1/grandse3vs1
p3vs1=(1-pnorm(abs(z3vs1),0,1))*2
p3vs1=round(p3vs1,3)

rownames(p2vs1)=rownames(p3vs1)=colnames(beta1)
colnames(p2vs1)="p 2vs1"
colnames(p3vs1)="p 3vs1"
or2vs1=round(exp(grandm2vs1),3)
or3vs1=round(exp(grandm3vs1),3)
library(pander)
pander(cbind(or2vs1,or3vs1,p2vs1,p3vs1))

# remove p>0.1 and repeat the process to get the reduced model

# code not show to save space

# calculate accuracy from imputed data

accudata=rbind(mice::complete(imp),mice::complete(imp,2),mice::complete(imp,3),mice::complete(imp,4),mice::complete(imp,5))
accudata1=select(accudata,X_traj_Group,anx,adltph,parity,n_cmdx,epdstota,gasscore00)
accudata1=mutate(accudata1,b2x=3.40757026+0.44856672*as.numeric(as.character(anx))+0.62986931*ifelse(adltph=="Yes",1,0)+0.13116528*parity+0.15869229*n_cmdx+0.05023554*epdstota-0.08887876*gasscore00,b3x=2.0240581+0.6739823*as.numeric(as.character(anx))+0.3173449*ifelse(adltph=="Yes",1,0)+0.5495376*parity+0.1418330*n_cmdx+0.1513239*epdstota-0.1014388*gasscore00,p1=1/(1+exp(b2x)+exp(b3x)),p2=exp(b2x)/(1+exp(b2x)+exp(b3x)),p3=1-p1-p2)

pred=apply(select(accudata1,p1,p2,p3),1,which.max)
table(accudata1$X_traj_Group,pred)/5

accuracy=(172.2+81.8+20.6)/507
accuracy

Re